## Antenna Position Measurement

The antennas are laid out on tiles, and these tiles are placed on site. Once this is done, a survey is needed to refine the positions of each antenna in the array.

Three reference posts are placed. The first is at the centre of the array, the second approximately 2.5 meters due north. The first and second post defines the $y$ axis. The third post is placed approximately 2.5 meters east of the centre, and reasonably close to at right angles to the $y$ axis.

The first reference point has coordinates (0,0). The second reference point has coordinates (0, y) and the third is not known, but must be established by measurement.

All measurements are made from the height of the antennas on the reference points.

In [1]:
import numpy as np
from scipy.optimize import minimize 

ImportError: No module named 'scipy'

In [ ]:
x0 = [0,0]
x1 = [0, 2209]

### Locating the third reference point

In [ ]:
d_0_2 = 2047
d_1_2 = 3020

def dist(a,b):
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

def f(x):
    return (dist(x0, x) - d_0_2)**2 + (dist(x1, x) - d_1_2)**2

initial_guess = [2047, 0]
res = minimize(f, initial_guess)
x2 = res.x

In [ ]:
reference_points = [x1, x0, x2]
reference_points

### Finding the antennas

This is done by measuring the distance from each antenna to the three reference points x0, x1 and x2.

In [ ]:
n_ant = 24
m = np.zeros((24,3))

In [ ]:
m[0,:] = [1563, 855, 2618]
m[1,:] = [1407, 825, 2355]
m[2,:] = [1750, 765, 2644]
m[3,:] = [839, 1373, 2416]
m[4,:] = [1151, 1422, 2986]
m[5,:] = [842, 1410, 2662]
m[6,:] = [2527, 1119, 929]
m[7,:] = [2274, 1200, 915]
m[8,:] = [2715, 1261, 824]
m[9,:] = [1684, 1064, 1457]
m[10,:] = [2238, 546, 1501]
m[11,:] = [1834, 805, 1493]
m[12,:] = [3320, 1111, 2370]
m[13,:] = [3385, 1192, 2131]
m[14,:] = [3446, 1247, 2555]
m[15,:] = [3063, 1048, 1531]
m[16,:] = [2760, 550, 2096]
m[17,:] = [2873, 784, 1689]
m[18,:] = [2342, 934, 2979]
m[19,:] = [2638, 1142, 3179]
m[20,:] = [2186, 993, 3020]
m[21,:] = [3130, 1260, 3140]
m[22,:] = [2545, 565, 2544]
m[23,:] = [2942, 1000, 2891]

In [ ]:
#Initial Guesses are from json
import requests
import json

pos_url = "https://tart.elec.ac.nz/dev/api/v1/imaging/antenna_positions"

def get_data(path):
    server = "https://tart.elec.ac.nz/dev"

    r = requests.get('{}/{}'.format(server, path))
    return json.loads(r.text)

def get_pos():
    return np.array(get_data('api/v1/imaging/antenna_positions'))

current_pos = get_pos()
current_pos


In [ ]:
initial_guess = np.zeros(2*n_ant)

for i in range(n_ant):
    initial_guess[2*i:2*i+2] = current_pos[i][0:2]*1000
    #print(current_pos[i][0:2]*1000)
initial_guess

pos_i = current_pos*1000
import matplotlib.pyplot as plt
plt.scatter(pos_i[:,0], pos_i[:,1])
plt.xlim(-2000,2000)
plt.ylim(-2000,2000)
plt.show()

In [ ]:
def f(x):
    ret = 0
    for i in range(n_ant):
        for j in range(3):
            p = [x[2*i],x[2*i+1]]
            ret += (dist(reference_points[j], p) - m[i,j])**2
    return ret

print(f(initial_guess))
res = minimize(f, initial_guess)
res

In [ ]:
pos = res.x.reshape((24,2))
pos

In [ ]:
plt.scatter(pos[:,0], pos[:,1])
plt.scatter(pos_i[:,0], pos_i[:,1])

plt.xlim(-2000,2000)
plt.ylim(-2000,2000)
plt.grid(True)
plt.show()

In [ ]:
result = np.zeros((n_ant, 3))
result[:,:-1] = np.round(pos/1000.0, 3)
json.dumps(result.tolist())